In [7]:
# 下载数据到data目录
import os
from kaggle.api.kaggle_api_extended import KaggleApi

# 初始化 API
api = KaggleApi()
api.authenticate()

# 下载竞赛数据集
api.competition_download_files("titanic", path="./data")


In [1]:
import pandas
test = pandas.read_csv("data/test.csv")
train = pandas.read_csv("data/train.csv")
train['Embarked'].fillna('S', inplace=True)
train.loc[train['Age'].isna(), 'Age'] = train['Age'].dropna().sample(n=train['Age'].isna().sum(), replace=True).values
test['Embarked'].fillna('S', inplace=True)
test.loc[test['Age'].isna(), 'Age'] = test['Age'].dropna().sample(n=test['Age'].isna().sum(), replace=True).values
features = ['Pclass','Sex', 'Age','SibSp','Parch','Fare','Embarked']
x_train = train[features]
x_test = test[features]
y_train = train['Survived']

/var/folders/n2/g_zrjq595h55vm_43q_b064m0000gn/T/ipykernel_29689/4211905262.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['Embarked'].fillna('S', inplace=True)
/var/folders/n2/g_zrjq595h55vm_43q_b064m0000gn/T/ipykernel_29689/4211905262.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always be

In [2]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=False)
x_train = dv.fit_transform(x_train.to_dict(orient='records'))
x_test = dv.fit_transform(x_test.to_dict(orient='records'))


In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
rfc = RandomForestClassifier()
from xgboost import XGBClassifier
xgbc = XGBClassifier()
print(cross_val_score(rfc, x_train, y_train,cv=5).mean())
print(cross_val_score(xgbc, x_train, y_train,cv=5).mean())

0.8002510827945516
0.7957504237022157


In [4]:
rfc.fit(x_train, y_train)
y_rfc_predict = rfc.predict(x_test)
rfc_sumbmission = pandas.DataFrame({'PassengerId': test['PassengerId'], "Survived": y_rfc_predict})
rfc_sumbmission.to_csv("data/rfc_submission.csv",index=False)

xgbc.fit(x_train, y_train)
y_xgbc_predict = xgbc.predict(x_test)
xgbc_sumbmission = pandas.DataFrame({'PassengerId': test['PassengerId'], "Survived": y_xgbc_predict})
xgbc_sumbmission.to_csv("data/xgbc_submission.csv",index=False)

In [6]:
from sklearn.model_selection import GridSearchCV
params = {'max_depth': range(2,7), 'n_estimators': range(100,1100,200), 'learning_rate': [0.05,0.1, 0.25,0.5,1.0]}
xgbc_bast = XGBClassifier()
gs = GridSearchCV(xgbc_bast, params, n_jobs=1,cv=5,verbose=1)
gs.fit(x_train, y_train)
y_gs_predict = gs.predict(x_test)
gs_sumbmission = pandas.DataFrame({'PassengerId': test['PassengerId'], "Survived": y_gs_predict})
gs_sumbmission.to_csv("data/xgbc_submission.csv",index=False)


Fitting 5 folds for each of 125 candidates, totalling 625 fits
